<a href="https://colab.research.google.com/github/Assaoka/Guide-to-Advanced-LLM-Techniques/blob/main/M%C3%B3dulo_3_Ensembles_de_LLMs_A_Sabedoria_das_Multid%C3%B5es.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Módulo 3: Ensembles de LLMs - A Sabedoria das Multidões

# Introdução

Nos módulos anteriores, focamos em como aprimorar a interação com um único modelo através da engenharia de prompt. Agora, vamos explorar uma técnica poderosa emprestada do aprendizado de máquina tradicional: os ensembles.

A ideia central de um ensemble é que "várias cabeças pensam melhor que uma". Em vez de confiar na resposta de um único modelo, combinamos as predições de múltiplos modelos (ou múltiplas predições do mesmo modelo) para chegar a uma decisão final mais robusta e precisa. LLMs, apesar de seu poder, podem ser suscetíveis a vieses ou alucinações. Usar um ensemble pode mitigar esses riscos e aumentar a confiabilidade geral do sistema.

# 2. Votação Majoritária

A votação é a forma mais intuitiva de ensemble. A ideia é consultar vários "especialistas" independentes e adotar a resposta da maioria. No contexto de LLMs, esses especialistas podem ser:
1. Diferentes Modelos: Você pode fazer a mesma pergunta para o Llama 3, Mixtral e Sabiá, e depois contar os votos de cada um.
2. Diferentes Prompts: Usar vários prompts (ex: um Zero-Shot, um Few-Shot) com o mesmo modelo.
3. Múltiplas Execuções do Mesmo Modelo: Esta é a abordagem conhecida como Self-Consistency (Autoconsistência), proposta por Wang et al. (2022) [1]. Executamos o mesmo prompt várias vezes com uma temperatura > 0 para gerar respostas diversas e escolhemos a mais frequente.

Vamos implementar a abordagem de Self-Consistency, pois é a mais prática quando se tem acesso a uma única API.

In [ ]:
!pip install -q langchain langchain-core langchain-community langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.1/131.1 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.7 MB/s eta 0:00:00


In [ ]:
from langchain_groq import ChatGroq
from google.colab import userdata

llm_groq = ChatGroq(
    model="gemma2-9b-it",
    api_key=userdata.get('GROQ_API_KEY'),
    temperature=0.7
)

In [ ]:
from typing import Literal
from pydantic import BaseModel, Field
from langchain_core.output_parsers import PydanticOutputParser

class ClassificacaoNoticia(BaseModel):
    emocao: Literal["Felicidade", "Tristeza", "Nojo", "Raiva",
                    "Medo", "Surpreza", "Desprezo"] = Field(
                    description="A emoção primária transmitida pelo texto. Estamos usando como padrão as emoções universais de ekman")

parser = PydanticOutputParser(pydantic_object=ClassificacaoNoticia)
formato = parser.get_format_instructions()


In [ ]:
from langchain_core.prompts import PromptTemplate

template = """Classifique a notícia abaixo quanto a emoção.
{format_instructions}
Notícia: {noticia}
"""

prompt = PromptTemplate.from_template(
    template=template,
    partial_variables={"format_instructions": formato}
)

In [ ]:
chain = prompt | llm_groq | parser
chain.invoke("Nesse cenário de instabilidade, os investidores não sabem o que fazer.")

ClassificacaoNoticia(emocao='Medo')

In [ ]:
def votacao(chain, texto: str, n: int) -> ClassificacaoNoticia:
    votos = [chain.invoke(texto) for _ in range(n)]
    vals = [voto.emocao for voto in votos]
    conts = {}
    for val in vals:
        print(val)
        if val in conts:
            conts[val] += 1
        else:
            conts[val] = 1

    return ClassificacaoNoticia(emocao=max(conts, key=conts.get))


votacao(chain, "Os resultados fiscais do primeiro semestre de 2024 indicam que, no curto prazo, a meta fiscal de -0,25% do PIB é mais viável, apesar das incertezas de médio e longo prazo devido aos desafios fiscais e ao aumento das despesas obrigatórias. O desempenho das receitas líquidas foi positivo, impulsionado pelas medidas legislativas de arrecadação, mas o crescimento das despesas, especialmente com benefícios previdenciários, elevou o déficit primário para R$ 68,7 bilhões, agravando o cenário fiscal. Apesar das medidas de contenção adotadas, a sustentabilidade das contas públicas a longo prazo permanece ameaçada, exigindo soluções concretas e uma postura cautelosa para os próximos anos.", 3)

Tristeza
Tristeza
Tristeza


ClassificacaoNoticia(emocao='Tristeza')

# 3. Negociação

E se, em vez de votar de forma independente, os "especialistas" pudessem debater e refinar suas ideias? Essa é a premissa da Negociação, uma técnica que simula um processo de argumentação para resolver discordâncias. O trabalho de Sun et al. (2023) [2] explora essa ideia, mostrando que um debate estruturado pode ajudar a resolver ambiguidades.

Vamos implementar um framework de debate iterativo com dois papéis:
1. Gerador: Gera a análise e classificação inicial.
2. Discriminador: Revisa a análise do propositor, busca falhas, pontos de vista alternativos ou ambiguidades e oferece uma contra-análise.

O processo é um loop:
1. O Gerador faz uma análise inicial.
2. O Discriminador avalia a análise. Podendo concordar ou apresentar uma contra proposta.
3. O Propositor revisa sua análise com base na crítica. E pode concordar ou enviar uma contra proposta.
4. O ciclo se repete até um número máximo de rodadas ou até que um consenso seja alcançado.

In [ ]:
gerador = ChatGroq(
    model="llama-3.3-70b-versatile",
    api_key=userdata.get('GROQ_API_KEY'),
    temperature=0.7
)

discriminador = ChatGroq(
    model="qwen-qwq-32b",
    api_key=userdata.get('GROQ_API_KEY'),
    temperature=0.7
)

In [ ]:
class Negociacao(BaseModel):
    raciocinio: str = Field(description="Linha de raciocínio passo a passo do modelo até chegar em uma decisão.")
    concordo: bool = Field(description="Representa se o modelo concorda ou não com o modelo anterior. Inicia como False.")
    emocao: Literal["Felicidade", "Tristeza", "Nojo", "Raiva",
                    "Medo", "Surpreza", "Desprezo"] = Field(
                    description="A emoção primária transmitida pelo texto. Estamos usando como padrão as emoções universais de ekman")

parser = PydanticOutputParser(pydantic_object=Negociacao)
formato = parser.get_format_instructions()

In [ ]:
gerador1_template = """Classifique a notícia abaixo quanto a emoção.
{format_instructions}
Notícia: {noticia}
"""

gerador1_prompt = PromptTemplate.from_template(
    template=gerador1_template,
    partial_variables={"format_instructions": formato}
)

In [ ]:
discriminador1_template = """Você está em um debate com outro agente. A tarefa de vocês é classificar a notícia abaixo quanto a emoção.
{format_instructions}
Notícia: {noticia}
Gerador: {gerador}
"""

discriminador1_prompt = PromptTemplate.from_template(
    template=discriminador1_template,
    partial_variables={"format_instructions": formato}
)

In [ ]:
iterações_template = """Você está em um debate com outro agente. A tarefa de vocês é classificar a notícia abaixo quanto a emoção.
{format_instructions}
Notícia: {noticia}
sua proposta: {analise}
Contra proposta: {contra_proposta}
"""

iterações_prompt = PromptTemplate.from_template(
    template=iterações_template,
    partial_variables={"format_instructions": formato}
)

In [ ]:
inicio = gerador1_prompt | gerador | parser
discriminar = discriminador1_prompt | discriminador | parser
iterações1 = iterações_prompt | gerador | parser
iterações2 = iterações_prompt | discriminador | parser

In [ ]:
def negociacao(noticia: str, n: int = 3):
    historico = [inicio.invoke({"noticia": noticia})]
    historico.append(discriminar.invoke({"noticia": noticia, "gerador": historico[-1].raciocinio}))
    for i in range(n-2):
        if i % 2 == 0:
            historico.append(iterações1.invoke({"noticia": noticia,
                                                "analise": historico[-2].raciocinio,
                                                "contra_proposta": historico[-1].raciocinio}))
        else:
            historico.append(iterações2.invoke({"noticia": noticia,
                                                "analise": historico[-2].raciocinio,
                                                "contra_proposta": historico[-1].raciocinio}))
        if historico[-1].concordo:
            break
    return historico


In [ ]:
negociacao("Os resultados fiscais do primeiro semestre de 2024 indicam que, no curto prazo, a meta fiscal de -0,25% do PIB é mais viável, apesar das incertezas de médio e longo prazo devido aos desafios")

OutputParserException: Invalid json output: <think>
Okay, I need to classify the given news article's emotion based on Ekman's basic emotions. The news is about the fiscal results for the first half of 2024 indicating that the short-term fiscal target of -0.25% of GDP is more viable, despite medium and long-term uncertainties due to challenges.

First, I'll analyze the content. The key points are the viability of the short-term target and the mention of uncertainties. The fiscal target being achievable in the short term might suggest a positive outlook, which could relate to happiness. However, the presence of "incertezas" (uncertainties) and "desafios" (challenges) in the medium to long term introduces a negative aspect. 

Now, considering Ekman's emotions: happiness is positive, but the uncertainties might evoke another emotion. The uncertainty and challenges could lead to worry or fear. Since the main focus is on the viability now but uncertainty ahead, fear (Medo) might be the prim